In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import pandas as pd

In [45]:
df = pd.read_csv("../data/taxi.csv", index_col=0)
display(df)

y = df["trip_duration"]
X = df.drop(["trip_duration"], axis=1)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.7)


df1 = pd.read_csv("../data/cs.csv", index_col=0).drop_duplicates()
unique = df1["map"].unique()
mapping = dict(zip(unique, np.arange(len(unique))))
df1 = df1.replace({"map": mapping})
df1["bomb_planted"] = df1["bomb_planted"].astype(int)
print(df1[df1["bomb_planted"] == 0].shape)
print(df1[df1["bomb_planted"] == 1].shape)

display(df1)

y = df1['bomb_planted']
X = df1.drop(['bomb_planted'], axis=1)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y, test_size=0.2, random_state = 4)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,hour_4_9,hour_10_15,hour_16_15,hour_night,pas0-3,pas4-6
0,-73.953918,40.778873,-73.963875,40.771164,400,0,0,1,0,1,0
1,-73.988312,40.731743,-73.994751,40.694931,1100,0,0,0,1,1,0
2,-73.997314,40.721458,-73.948029,40.774918,1635,0,0,1,0,1,0
3,-73.961670,40.759720,-73.956779,40.780628,1141,1,0,0,0,0,1
4,-74.017120,40.708469,-73.988182,40.740631,848,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
729317,-73.965919,40.789780,-73.952637,40.789181,296,0,1,0,0,1,0
729318,-73.996666,40.737434,-74.001320,40.731911,315,0,0,0,1,1,0
729319,-73.997849,40.761696,-74.001488,40.741207,673,0,0,1,0,1,0
729320,-74.006706,40.708244,-74.013550,40.713814,447,1,0,0,0,1,0


(12946, 16)
(13207, 16)


,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
48656,174.94,4.0,14.0,0,0,500.0,500.0,0.0,269.0,25750.0,27400.0,0.0,3.0,0.0,5.0,5.0
111472,175.00,0.0,0.0,1,0,500.0,500.0,300.0,400.0,1300.0,600.0,0.0,0.0,1.0,5.0,5.0
62915,94.94,5.0,8.0,2,0,500.0,467.0,500.0,387.0,9450.0,200.0,4.0,4.0,4.0,5.0,5.0
19132,175.00,14.0,12.0,3,0,500.0,500.0,0.0,497.0,16200.0,12650.0,0.0,5.0,0.0,5.0,5.0
90843,175.00,0.0,4.0,1,0,500.0,500.0,100.0,387.0,17150.0,24000.0,0.0,4.0,0.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122390,12.26,10.0,13.0,2,1,180.0,0.0,197.0,0.0,1150.0,0.0,1.0,0.0,1.0,2.0,0.0
122399,20.27,11.0,13.0,2,1,174.0,239.0,200.0,284.0,2000.0,2200.0,2.0,3.0,2.0,2.0,3.0
122400,0.24,11.0,13.0,2,1,119.0,178.0,182.0,195.0,2300.0,950.0,2.0,2.0,2.0,2.0,2.0
122404,35.41,11.0,14.0,2,1,315.0,428.0,382.0,467.0,150.0,5700.0,2.0,5.0,2.0,4.0,5.0


In [51]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(10,)),
        tf.keras.layers.Dense(32, activation="relu"),
        #tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(16, activation="relu"),
        #tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(8, activation="relu"),
        #tf.keras.layers.Dropout(0.01),
        tf.keras.layers.Dense(1, activation="linear")
    ]
)

In [52]:
# стохастический градиентного спуск
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mae")

In [53]:
model_regression.fit(X_train_reg, y_train_reg, epochs=15)

Epoch 1/15
1965/1965 [==============================] - 4s 1ms/step - loss: 344.6070
Epoch 2/15
1965/1965 [==============================] - 3s 1ms/step - loss: 335.9182
Epoch 3/15
1965/1965 [==============================] - 3s 1ms/step - loss: 335.6509
Epoch 4/15
1965/1965 [==============================] - 3s 1ms/step - loss: 335.5477
Epoch 5/15
1965/1965 [==============================] - 3s 2ms/step - loss: 335.4981
Epoch 6/15
1965/1965 [==============================] - 3s 2ms/step - loss: 335.2471
Epoch 7/15
1965/1965 [==============================] - 3s 2ms/step - loss: 335.2003
Epoch 8/15
1965/1965 [==============================] - 3s 2ms/step - loss: 335.3406
Epoch 9/15
1965/1965 [==============================] - 3s 1ms/step - loss: 335.3064
Epoch 10/15
1965/1965 [==============================] - 3s 1ms/step - loss: 335.1892
Epoch 11/15
1965/1965 [==============================] - 3s 2ms/step - loss: 335.2377
Epoch 12/15
1965/1965 [==============================] - 3s 2ms

In [54]:
y_pred_reg = model_regression.predict(X_test_reg)

17684/17684 [==============================] - 21s 1ms/step


In [55]:
print(mean_absolute_error(y_test_reg, y_pred_reg))
print(mean_squared_error(y_test_reg, y_pred_reg))
print(r2_score(y_test_reg, y_pred_reg))
print(max_error(y_test_reg, y_pred_reg))

335.43032382978936
217257.98971004787
-0.03921677163174375
7258.798583984375


In [217]:
model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(15,)),
#         tf.keras.layers.Dense(20, activation="relu"),
#         tf.keras.layers.Dense(10, activation="tanh"),
#         tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(3, activation="tanh"),
#         tf.keras.layers.Dense(32, activation="tanh"),
#         tf.keras.layers.Dense(16, activation="tanh"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ]
)

In [218]:
model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy")

In [219]:
model_classification.fit(X_train_clf, y_train_clf, epochs=25)

Epoch 1/25
654/654 [==============================] - 1s 1ms/step - loss: 0.7004
Epoch 2/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6958
Epoch 3/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6928
Epoch 4/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6908
Epoch 5/25
654/654 [==============================] - 1s 2ms/step - loss: 0.6891
Epoch 6/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6876
Epoch 7/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6863
Epoch 8/25
654/654 [==============================] - 1s 2ms/step - loss: 0.6851
Epoch 9/25
654/654 [==============================] - 1s 2ms/step - loss: 0.6835
Epoch 10/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6825
Epoch 11/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6816
Epoch 12/25
654/654 [==============================] - 1s 1ms/step - loss: 0.6811
Epoch 13/25
654/654 [====

In [220]:
model_classification.predict(X_test_clf, verbose=None)[:5]

array([[0.44349626],
       [0.67266536],
       [0.44349626],
       [0.61967945],
       [0.44349626]], dtype=float32)

In [221]:
y_pred = np.around(model_classification.predict(X_test_clf, verbose=None))
print(classification_report(y_test_clf, y_pred))
print(confusion_matrix(y_test_clf, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.65      0.60      2624
           1       0.58      0.50      0.54      2607

    accuracy                           0.57      5231
   macro avg       0.57      0.57      0.57      5231
weighted avg       0.57      0.57      0.57      5231

[[1695  929]
 [1305 1302]]


In [236]:
class Relu():

    def callFunction(x):
        return np.maximum(0,x)

    def callDerivative(x):
        return np.where(x > 0, 1.0, 0.0)

class Sigmoid(Activation):

    def callFunction(x):
        return 1 / (1 + np.exp(-x))

    def callDerivative(x):
        return Sigmoid.callFunction(x) * (1- Sigmoid.callFunction(x))
class TangH(Activation):
    
    def callFunction(x):
        #return 1/(1+np.exp(-x))
        return (np.exp(x) - np.exp(-x)) / (np.exp(x)+np.exp(-x))

    def callDerivative(x):
        return 1 - TangH.callFunction(x) ** 2

In [237]:
class FCLayer():
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error 

class ActivationLayer():
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [238]:
class MeanSquaredError():
    def fun(y_true, y_pred):
        return np.mean(np.power(y_true-y_pred, 2))
    
    def derivative(y_true, y_pred):
        return 2*(y_pred-y_true)/y_true.size

In [239]:
class NN:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def compile(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(*output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):

        samples = len(x_train)

        for i in range(epochs):
            print("epoch ", i+1)
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

In [263]:
X_clf = df1.drop(["bomb_planted"], axis=1).values
std = StandardScaler()
std.fit(X_clf)
X_clf = std.transform(X_clf)
X_clf= X_clf.reshape(X_clf.shape[0], 1, X_clf.shape[1])
print(X_clf.shape)
y_clf = df1["bomb_planted"].values

(26153, 1, 15)


In [241]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf,
                                                                        y_clf,
                                                                        stratify=y_clf,
                                                                        test_size=0.2)

In [249]:
model_classification = NN()
model_classification.add(FCLayer(15, 10))
model_classification.add(ActivationLayer(TangH.callFunction, TangH.callDerivative))
model_classification.add(FCLayer(10, 5))
model_classification.add(ActivationLayer(TangH.callFunction, TangH.callDerivative))
model_classification.add(FCLayer(5, 1))
model_classification.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
model_classification.compile(MeanSquaredError.fun, MeanSquaredError.derivative)

In [250]:
model_classification.fit(X_clf_train, y_clf_train, epochs=15, learning_rate=0.001)

epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
epoch  11
epoch  12
epoch  13
epoch  14
epoch  15


In [253]:
pred_classification = model_classification.predict(X_clf_test)

In [254]:
print(classification_report(y_clf_test, np.around(pred_classification)))
print(confusion_matrix(y_clf_test, np.around(pred_classification)))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94      2589
           1       0.91      0.99      0.95      2642

    accuracy                           0.94      5231
   macro avg       0.95      0.94      0.94      5231
weighted avg       0.95      0.94      0.94      5231

[[2333  256]
 [  36 2606]]


In [274]:
y_reg = df["trip_duration"].values
X_reg = df.drop(["trip_duration"], axis=1).values.reshape(df.shape[0], 1, df.shape[1] - 1)
X_reg.shape
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.7,random_state=4)

In [275]:
model_reg = NN()
model_reg.add(FCLayer(10, 6))
model_reg.add(ActivationLayer(Relu.callFunction, Relu.callFunction))
model_reg.add(FCLayer(6, 4))
model_reg.add(ActivationLayer(Relu.callFunction, Relu.callFunction))
model_reg.add(FCLayer(4, 1))
model_reg.compile(MeanSquaredError.fun, MeanSquaredError.derivative)

In [276]:
model_reg.fit(X_train_reg, y_train_reg, epochs=15, learning_rate=0.001)

epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
epoch  11
epoch  12
epoch  13
epoch  14
epoch  15


In [277]:
pred_reg = model_reg.predict(X_test_reg)

In [278]:
print(mean_absolute_error(y_test_reg, pred_reg))
print(mean_squared_error(y_test_reg, pred_reg))
print(r2_score(y_test_reg, pred_reg))
print(max_error(y_test_reg, pred_reg))

349.6852361219175
209122.76846958912
-0.0013599867103781804
7105.088016306821
